### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
!wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-04-01 07:39:57--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.216.207.163
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.216.207.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-04-01 07:39:57 (141 MB/s) - ‘iris.csv.1’ saved [4609/4609]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data, test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [6]:
label_uji = data_uji.pop('species')

In [7]:
data_uji.head()

,sepal_length,sepal_width,petal_length,petal_width
146,6.3,2.5,5.0,1.9
28,5.2,3.4,1.4,0.2
13,4.3,3.0,1.1,0.1
100,6.3,3.3,6.0,2.5
49,5.0,3.3,1.4,0.2


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data. 

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [8]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key] = prior[key]/n_data
  return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [9]:
prior = hitung_prior(data_latih['species'])
print(prior)

Counter({'Iris-setosa': 0.35, 'Iris-virginica': 0.3416666666666667, 'Iris-versicolor': 0.30833333333333335})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [10]:
def hitung_rata2_std_kelas(input_data):
  list_columns = input_data.columns[:-1]
  class_column_name = input_data.columns[-1]
  list_class = set(input_data[class_column_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
      rata2[(a_class,column)] = input_data.loc[input_data[class_column_name]== a_class][column].mean()
      std[(a_class,column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
  return(rata2,std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [11]:
import math
def hitung_likelihood_gaussian(data,rata2,std):
  hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas. 

In [12]:
def training_naive_bayes_gaussian(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  (rata2,std) = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [13]:
model = training_naive_bayes_gaussian(data_latih)

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [14]:
def testing_naive_bayes_gaussian(model,data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class,1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class]*hitung_likelihood_gaussian(data_uji[column],rata2[(a_class,column)],std[(a_class,column)])
    posterior[a_class] = posterior[a_class] * prior[a_class]
  kelas_uji = max(posterior,key=posterior.get)
  return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [15]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Iris-virginica
Iris-virginica


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [16]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model,data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total == label_uji))

Total prediksi benar:  27


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [17]:
!wget https://dataset-ppm.s3.amazonaws.com/car_sample.csv

--2021-04-01 07:40:10--  https://dataset-ppm.s3.amazonaws.com/car_sample.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.217.195.81
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.217.195.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7226 (7.1K) [text/csv]
Saving to: ‘car_sample.csv.1’

car_sample.csv.1    100%[===================>]   7.06K  --.-KB/s    in 0s      

2021-04-01 07:40:11 (140 MB/s) - ‘car_sample.csv.1’ saved [7226/7226]



In [18]:
import pandas as pd
data_tugas = pd.read_csv('car_sample.csv')
data_tugas.head()

,buying,maint,lug_boot,safety,class
0,vhigh,low,small,high,acc
1,vhigh,low,big,high,acc
2,vhigh,med,big,med,acc
3,med,med,med,high,acc
4,low,high,med,high,acc


In [19]:
#Melihat data secara keseluruhan
data_tugas

,buying,maint,lug_boot,safety,class
0,vhigh,low,small,high,acc
1,vhigh,low,big,high,acc
2,vhigh,med,big,med,acc
3,med,med,med,high,acc
4,low,high,med,high,acc
...,...,...,...,...,...
255,low,high,big,high,vgood
256,low,med,big,high,vgood
257,low,high,big,high,vgood
258,low,med,med,high,vgood


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [20]:
from sklearn.model_selection import train_test_split
#Menggunakan train_test_split untuk membagi data latih dan data uji dengan rasio 70:30
tugas_latih,tugas_uji = train_test_split(data_tugas,test_size=0.3)

In [21]:
print(tugas_uji.shape[0])
print(tugas_latih.shape[0])

78
182


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [22]:
#category dari data_uji bernama class
tugas_label_uji = tugas_uji.pop('class')

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [23]:
prior_tugas = hitung_prior(tugas_latih['class'])
prior_tugas

Counter({'acc': 0.27472527472527475,
         'good': 0.25274725274725274,
         'unacc': 0.2692307692307692,
         'vgood': 0.2032967032967033})

Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [24]:
def hitung_likelihood_multinomial(data_latih):
  #tulis kode Anda disini
  list_columns = data_latih.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = set(data_latih[class_column_name])
  result = {}
  for kolom in list_columns:
    for a_class in list_class:
      unique = set(data_latih[kolom])
      for value in unique:
        count_wc = data_latih.loc[(data_latih[kolom] == value) & (data_latih[class_column_name]==a_class)][kolom].count()
        count_c = data_latih.loc[data_latih[class_column_name]==a_class][kolom].count()
        result[(kolom,value,a_class)]=count_wc/count_c
  return result

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [ ]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
likelihood

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [26]:
#Melihat data latih unik
tugas_latih['class'].unique()

array(['acc', 'vgood', 'good', 'unacc'], dtype=object)

In [42]:
def training_naive_bayes_multinomial(tugas_latih):
  #tulis kode Anda disini
  kolom_kategori = tugas_latih.columns[-1]

  nilai_prior = hitung_prior(tugas_latih[kolom_kategori])
  likelihood = hitung_likelihood_multinomial(tugas_latih)
  list_class = set(tugas_latih[kolom_kategori])
  list_columns = tugas_latih.columns[:-1]
  model = {}
  
  model['nilai_prior'] = nilai_prior
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  model['likelihood'] = likelihood
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [ ]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)
model_tugas

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [45]:
def testing_naive_bayes_multinomial(model,tugas_uji):
  #tulis kode Anda disini
  #mendapatkan category dari tugas_uji
  prior = model['nilai_prior']
  list_class = model['list_class']
  list_columns = model['list_columns']
  likelihood = model['likelihood']
  posterior = dict.fromkeys(list_class,1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class]*likelihood[column, tugas_uji[column],a_class]
    posterior[a_class] = posterior[a_class] * prior[a_class]
  kelas_uji = max(posterior,key=posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [ ]:
model_tugas['likelihood']

In [47]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas, uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

acc
acc


Lakukan pengujian terhadap semua data uji.

In [48]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  65


In [49]:
len(tugas_label_uji)

78